In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os
from src.investing import Investing
import pandas as pd
import numpy as np
reverse = False

In [11]:
def check_miss_file(symbol):
    df = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    QUARTER = ['Q1', 'Q2', 'Q3', 'Q4']
    df_temp = df[['download_Q1', 'download_Q2', 'download_Q3', 'download_Q4']]
    rows, cols = np.where(df_temp.isna())
    lst_time, lst_quarter = [],  []
    for row, col in zip(rows, cols):
        if df[f'Link_{QUARTER[col]}'][row] != '[]' and pd.isna(df[f'Link_{QUARTER[col]}'][row]) == False:
            lst_time.append(df['Year'][row])
            lst_quarter.append(QUARTER[col])

    df_new = pd.DataFrame({
                'Symbol': symbol,
                    'Year':lst_time,
                'Quarter': lst_quarter,
                })
    return df_new

def check_miss_all(reverse = True):
    df_all = pd.read_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv')
    df_miss = pd.DataFrame(columns=['Symbol', 'Year', 'Quarter'])
    for id in df_all['Symbol'].index:
        symbol = df_all['Symbol'][id]
        if os.path.exists(fr'Data\{symbol}\docs\check.csv'):
            df_temp = check_miss_file(symbol)
            df_miss = pd.concat([df_miss, df_temp])
            df_all['check'][id] = 'Done'
    df_miss.to_csv(f'docs\miss_{reverse}.csv', index=False)
    df_all.to_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv', index=False)

check_miss_all(reverse = False)

In [ ]:
def get_link(symbol, year, quarter):
    df_check = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    print(df_check[f'Link_{quarter}'][df_check['Year'] == year])
    links = eval(df_check[f'Link_{quarter}'][df_check['Year'] == year].iloc[0])
    times = eval(df_check[f'Time_{quarter}'][df_check['Year'] == year].iloc[0])
    for i, time in enumerate(times):
        if "(訂正)" not in time:
            return links[i], time
        
def thread_file():
    df_miss = pd.read_csv(f'docs\miss_{reverse}.csv')
    id = df_miss[df_miss['check'].isna()].index[0]
    symbol = int(df_miss['Symbol'][id])
    year = int(df_miss['Year'][id])
    quarter = df_miss['Quarter'][id]
    print(symbol, year, quarter)
    df_miss['check'][id] = 'Doing'
    df_miss.to_csv(f'docs\miss_{reverse}.csv', index=False)
    link, time = get_link(symbol, year, quarter)
    return link, time

thread_file()